In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math


In [4]:
import random
import numpy as np
from numpy import ndarray

In [67]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [6]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [7]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (SB, MQ), (SB, MQ), (SB, SQ), (LB, SQ), (SB, MQ), (SB, MQ), (SB, LQ),
     (MB, SQ), (SB, SQ), (MB, MQ), (SB, SQ), (SB, LQ), (MB, SQ), (MB, LQ), (MB, MQ)
 ]

In [78]:
# Compute total number of required points across all test combinations
required_points = sum(data_size for data_size, _ in test_combinations)

# Sample from the full dataset without replacement
if required_points > total_points:
    raise ValueError(f"Need {required_points} points but only {total_points} available.")

sampled_indices = np.random.choice(total_points, size=required_points, replace=False)
combined_dataset = dataset[sampled_indices]


In [79]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


In [ ]:
print(len(queries_list))
print(len(combined_dataset))

15
1590000


**-------------Cover Tree---------------**

**Izbicki's implementation**

In [ ]:
import numpy as np
from typing import Optional, List, Tuple
from dataclasses import dataclass

@dataclass
class Node:
    point: np.ndarray
    level: int
    children: List['Node']
    _maxdist_cache: Optional[float] = None  # Cache for maxdist

    @property
    def covdist(self) -> float:
        """Coverage distance of the node."""
        return 2.0 ** self.level

    @property
    def maxdist(self) -> float:
        """Maximum distance within the subtree rooted at this node."""
        if self._maxdist_cache is None:
            if not self.children:
                self._maxdist_cache = 0
            else:
                self._maxdist_cache = max(
                    self.covdist,
                    max(child.maxdist + np.linalg.norm(self.point - child.point)
                        for child in self.children)
                )
        return self._maxdist_cache

    def update_maxdist(self):
        """Reset and recompute maxdist cache."""
        self._maxdist_cache = None
        _ = self.maxdist  # Force recomputation

class CoverTree:
    def __init__(self):
        self.root: Optional[Node] = None

    def insert(self, x: np.ndarray) -> None:
        """Insert a point into the cover tree."""
        if self.root is None:
            self.root = Node(x, 0, [])
            return

        d = np.linalg.norm(self.root.point - x)
        if d > self.root.covdist:
            # Precompute levels to expand
            levels_to_expand = int(np.ceil(np.log2(d / self.root.covdist)))
            for _ in range(levels_to_expand):
                self.root = Node(self.root.point, self.root.level + 1, [self.root])
            self.root = Node(x, self.root.level + 1, [self.root])
            return

        self._insert_recursive(self.root, x)

    def _insert_recursive(self, p: Node, x: np.ndarray) -> bool:
        """Recursive insertion into the cover tree."""
        for child in p.children:
            if np.linalg.norm(child.point - x) <= child.covdist:
                if self._insert_recursive(child, x):
                    p.update_maxdist()
                    return True

        # If x fits within the current node's coverage
        if np.linalg.norm(p.point - x) <= p.covdist:
            p.children.append(Node(x, p.level - 1, []))
            p.update_maxdist()
            return True

        return False

    def find_nearest(self, x: np.ndarray) -> Tuple[np.ndarray, float]:
        """Find the nearest neighbor to the query point."""
        if self.root is None:
            raise ValueError("Tree is empty")

        return self._find_nearest_recursive(self.root, x, self.root)

    def _find_nearest_recursive(self, p: Node, x: np.ndarray, best: Node) -> Tuple[np.ndarray, float]:
        """Recursive nearest neighbor search with pruning."""
        d_p = np.linalg.norm(p.point - x)
        d_best = np.linalg.norm(best.point - x)

        # Update the best point if the current node is closer
        if d_p < d_best:
            best = p
            d_best = d_p

        # Sort children by distance to the query point
        for child in sorted(p.children, key=lambda c: np.linalg.norm(c.point - x)):
            d_x_to_child = np.linalg.norm(child.point - x)

            # Prune subtrees based on the pruning condition
            if d_best > d_x_to_child - child.maxdist:
                child_point, child_dist = self._find_nearest_recursive(child, x, best)
                if child_dist < d_best:
                    best = Node(child_point, p.level, [])
                    d_best = child_dist

        return best.point, d_best


---------

**----------CV Run 1----------------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 737.73it/s]


Build Time: 1.56 sec, Search Time: 0.13 sec

1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 542.26it/s]


Build Time: 2.15 sec, Search Time: 0.18 sec

2: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 45.45 sec, Search Time: 0.19 sec

13: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:07<00:00, 69.67it/s]


Build Time: 44.92 sec, Search Time: 7.12 sec

14: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 64.25it/s]

Build Time: 45.49 sec, Search Time: 1.54 sec

=== Summary of CV Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 1.56 sec, Search: 0.13 sec
Data:   10000 points, Queries:  100 => Build: 2.15 sec, Search: 0.18 sec
Data:   10000 points, Queries:   20 => Build: 2.35 sec, Search: 0.05 sec
Data: 1000000 points, Queries:   20 => Build: 364.89 sec, Search: 0.17 sec
Data:   10000 points, Queries:  100 => Build: 4.52 sec, Search: 1.44 sec
Data:   10000 points, Queries:  100 => Build: 4.85 sec, Search: 1.41 sec
Data:   10000 points, Queries:  500 => Build: 4.14 sec, Search: 6.24 sec
Data:  100000 points, Queries:   20 => Build: 42.33 sec, Search: 0.26 sec
Data:   10000 points, Queries:   20 => Build: 4.31 sec, Search: 0.24 sec
Data:  100000 points, Queries:  100 => Build: 43.43 sec, Search: 1.27 sec
Data:   10000 points, Queries:   20 => Build: 4.41 sec, Search: 0.25 sec
Data:   10000 points, Queries:  500 => Build: 4.47 sec, Search: 6.67 sec
Data:  100000 points, Queries:   20 => Bu

**----------CV Run 2-----------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 588.13it/s]


Build Time: 3.73 sec, Search Time: 0.17 sec

1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 486.62it/s]


Build Time: 2.34 sec, Search Time: 0.20 sec

2: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 48.67 sec, Search Time: 0.38 sec

13: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:07<00:00, 69.70it/s]


Build Time: 47.58 sec, Search Time: 7.11 sec

14: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 59.98it/s]

Build Time: 48.13 sec, Search Time: 1.65 sec

=== Summary of CV Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 3.73 sec, Search: 0.17 sec
Data:   10000 points, Queries:  100 => Build: 2.34 sec, Search: 0.20 sec
Data:   10000 points, Queries:   20 => Build: 2.45 sec, Search: 0.05 sec
Data: 1000000 points, Queries:   20 => Build: 407.65 sec, Search: 0.26 sec
Data:   10000 points, Queries:  100 => Build: 4.46 sec, Search: 1.24 sec
Data:   10000 points, Queries:  100 => Build: 5.56 sec, Search: 1.20 sec
Data:   10000 points, Queries:  500 => Build: 4.54 sec, Search: 6.49 sec
Data:  100000 points, Queries:   20 => Build: 46.07 sec, Search: 0.23 sec
Data:   10000 points, Queries:   20 => Build: 4.69 sec, Search: 0.33 sec
Data:  100000 points, Queries:  100 => Build: 47.38 sec, Search: 1.62 sec
Data:   10000 points, Queries:   20 => Build: 4.87 sec, Search: 0.21 sec
Data:   10000 points, Queries:  500 => Build: 5.04 sec, Search: 7.24 sec
Data:  100000 points, Queries:   20 => Bu

**--------CV Run  3-----------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 617.49it/s]


Build Time: 4.10 sec, Search Time: 0.16 sec

1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 508.82it/s]


Build Time: 2.49 sec, Search Time: 0.19 sec

2: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 49.55 sec, Search Time: 0.25 sec

13: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:07<00:00, 67.29it/s]


Build Time: 49.56 sec, Search Time: 7.36 sec

14: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 76.43it/s]

Build Time: 49.81 sec, Search Time: 1.30 sec

=== Summary of CV Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 4.10 sec, Search: 0.16 sec
Data:   10000 points, Queries:  100 => Build: 2.49 sec, Search: 0.19 sec
Data:   10000 points, Queries:   20 => Build: 2.58 sec, Search: 0.05 sec
Data: 1000000 points, Queries:   20 => Build: 421.51 sec, Search: 0.26 sec
Data:   10000 points, Queries:  100 => Build: 4.72 sec, Search: 1.27 sec
Data:   10000 points, Queries:  100 => Build: 4.61 sec, Search: 1.21 sec
Data:   10000 points, Queries:  500 => Build: 4.82 sec, Search: 5.79 sec
Data:  100000 points, Queries:   20 => Build: 48.62 sec, Search: 0.25 sec
Data:   10000 points, Queries:   20 => Build: 4.94 sec, Search: 0.24 sec
Data:  100000 points, Queries:  100 => Build: 48.06 sec, Search: 1.31 sec
Data:   10000 points, Queries:   20 => Build: 4.87 sec, Search: 0.23 sec
Data:   10000 points, Queries:  500 => Build: 4.92 sec, Search: 6.22 sec
Data:  100000 points, Queries:   20 => Bu

**-------CV Analysis----------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 646.44',
    'Total Time: 711.56',
    'Total Time: 731.23'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 696.4100 sec, Std Dev: 44.3788 sec, CV: 6.37%


--------------

**--------------------------VP_MWV-----------------------------**

In [ ]:
import numpy as np

class VPNode:
    def __init__(self, vp=None):
        # Vantage point
        self.vp = vp

        # Distance bounds
        self.low_near = float('inf')
        self.low_far = 0
        self.high_near = float('inf')
        self.high_far = 0

        # Child nodes
        self.lowChild = None
        self.highChild = None

        # Bucket for leaf nodes
        self.bucket = []

    def is_leaf(self):
        return self.lowChild is None and self.highChild is None

class VPTree:
    def __init__(self, bucket_size=50, distance_fn=None):
        """
        Initialize a VP-Tree

        Parameters:
        - bucket_size: Square root of maximum bucket size (b where b² is max bucket size)
        - distance_fn: Distance function to use (defaults to Euclidean)
        """
        self.bucket_size = bucket_size
        self.root = None

        # Default to Euclidean distance if none provided
        self.distance_fn = distance_fn if distance_fn is not None else self._euclidean_distance

    def _euclidean_distance(self, x, y):
        """Calculate Euclidean distance between two points"""
        x = np.asarray(x)
        y = np.asarray(y)
        diff = x - y
        return np.sqrt(np.dot(diff, diff))

    def insert(self, x):
        """Insert a new datapoint x into the VP-tree"""
        # If tree is empty, create root with this point as VP
        if self.root is None:
            self.root = VPNode(vp=x)
            return

        p = self.root

        # Traverse until we reach a leaf node
        while not p.is_leaf():
            # Calculate distance from x to current vantage point
            dist = self.distance_fn(x, p.vp)

            # Determine which child to go to
            if dist < (p.low_far + p.high_near) / 2:
                # Update distance bounds
                p.low_far = max(dist, p.low_far)
                p.low_near = min(dist, p.low_near)

                # Go to low (near) child
                p = p.lowChild
            else:
                # Update distance bounds
                p.high_far = max(dist, p.high_far)
                p.high_near = min(dist, p.high_near)

                # Go to high (far) child
                p = p.highChild

        # Add x to bucket leaf node p
        p.bucket.append(x)

        # If bucket exceeds b², split it
        if len(p.bucket) > self.bucket_size**2:
            self._split_node(p)

    def _select_vantage_point(self, points):
        """Select a vantage point from the bucket
        Can use various methods; simple random selection is shown here.
        """
        # For simplicity, using the first point as VP
        # Could implement more sophisticated selection methods
        if points:
            return points[0]
        return None

    def _split_node(self, p):
        """Split a node using the CART-inspired weighted variance method"""
        points = p.bucket.copy()

        # Select vantage point from bucket
        vp = self._select_vantage_point(points)
        points.remove(vp)  # Remove VP from points list

        # Calculate distances to vantage point
        distances = [(self.distance_fn(vp, point), point) for point in points]

        # Sort by distance
        distances.sort(key=lambda x: x[0])
        sorted_points = [point for _, point in distances]
        sorted_dists = [dist for dist, _ in distances]

        # Find optimal split using weighted variance minimization
        n = len(sorted_points)
        best_split = 0
        min_weighted_variance = float('inf')

        for s in range(1, n):
            # Calculate variances for 1:s and s:n
            if s > 0:
                var_left = np.var(sorted_dists[:s]) if s > 1 else 0
            else:
                var_left = 0

            if n-s > 0:
                var_right = np.var(sorted_dists[s:]) if n-s > 1 else 0
            else:
                var_right = 0

            # Calculate weighted variance
            weighted_var = s * var_left + (n-s) * var_right

            if weighted_var < min_weighted_variance:
                min_weighted_variance = weighted_var
                best_split = s

        # Create new children nodes
        p.lowChild = VPNode(vp=vp)
        p.highChild = VPNode(vp=self._select_vantage_point(sorted_points[best_split:]))

        # Distribute points to children based on best split
        p.lowChild.bucket = sorted_points[:best_split]
        p.highChild.bucket = sorted_points[best_split:]

        # Initialize distance bounds
        if p.lowChild.bucket:
            low_dists = sorted_dists[:best_split]
            p.low_near = min(low_dists)
            p.low_far = max(low_dists)

        if p.highChild.bucket:
            # If we have a vantage point for the high child, calculate distances to it
            high_dists = [self.distance_fn(p.highChild.vp, point) for point in p.highChild.bucket]
            p.high_near = min(high_dists) if high_dists else float('inf')
            p.high_far = max(high_dists) if high_dists else 0

        # Clear the bucket as it's no longer a leaf
        p.bucket = []

    def nearest_neighbor(self, query):
        """Optimized nearest neighbor search with batch processing for bucket points"""
        best_dist = float('inf')
        best_point = None

        # Use a stack instead of recursion to reduce overhead
        # Each entry is (node, min_possible_dist)
        stack = [(self.root, 0)]

        while stack:
            node, min_dist = stack.pop()

            # Skip this node if it cannot contain a better solution
            if min_dist >= best_dist:
                continue

            # Check vantage point distance
            if node.vp is not None:
                # Calculate distance to vantage point
                diff = np.subtract(query, node.vp)
                dist_to_vp = np.sqrt(np.dot(diff, diff))

                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.vp

            # If leaf node, check all points in bucket using batch processing
            if node.is_leaf():
                if node.bucket:
                    # Convert bucket to numpy array once
                    bucket_array = np.array(node.bucket)

                    # Vectorized distance calculation
                    query_array = np.array(query)
                    diffs = bucket_array - query_array
                    squared_dists = np.sum(diffs**2, axis=1)

                    # Find minimum distance point without unnecessary sqrt operations
                    min_idx = np.argmin(squared_dists)
                    min_squared_dist = squared_dists[min_idx]

                    # Only compute sqrt for potentially better distance
                    if min_squared_dist < best_dist**2:
                        min_dist = np.sqrt(min_squared_dist)
                        if min_dist < best_dist:
                            best_dist = min_dist
                            best_point = node.bucket[min_idx]
                continue

            # Calculate threshold
            threshold = (node.low_far + node.high_near) / 2

            # Calculate bounds for both children
            if dist_to_vp < threshold:
                # Query is in near region
                # Add far child to stack first (so near child gets processed first)
                if node.highChild:
                    far_bound = max(0, dist_to_vp - node.low_far)
                    if far_bound < best_dist:
                        stack.append((node.highChild, far_bound))

                # Add near child with zero bound (must contain query)
                if node.lowChild:
                    stack.append((node.lowChild, 0))
            else:
                # Query is in far region
                # Add near child to stack first (so far child gets processed first)
                if node.lowChild:
                    near_bound = max(0, node.high_near - dist_to_vp)
                    if near_bound < best_dist:
                        stack.append((node.lowChild, near_bound))

                # Add far child with zero bound (must contain query)
                if node.highChild:
                    stack.append((node.highChild, 0))

        return best_dist, best_point

----------

**VP_MWV (Run 1)**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 1093.50it/s]


Build Time: 2.08 sec, Search Time: 0.09 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 791.49it/s]


Build Time: 1.56 sec, Search Time: 0.12 sec

3: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 26.79 sec, Search Time: 0.03 sec

14: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 650.41it/s]


Build Time: 21.78 sec, Search Time: 0.76 sec

15: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 571.04it/s]

Build Time: 22.16 sec, Search Time: 0.17 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 2.08 sec, Search: 0.09 sec
Data:   10000 points, Queries:  100 => Build: 1.56 sec, Search: 0.12 sec
Data:   10000 points, Queries:   20 => Build: 2.13 sec, Search: 0.02 sec
Data: 1000000 points, Queries:   20 => Build: 246.68 sec, Search: 0.12 sec
Data:   10000 points, Queries:  100 => Build: 4.42 sec, Search: 0.15 sec
Data:   10000 points, Queries:  100 => Build: 1.50 sec, Search: 0.16 sec
Data:   10000 points, Queries:  500 => Build: 2.79 sec, Search: 0.79 sec
Data:  100000 points, Queries:   20 => Build: 24.72 sec, Search: 0.04 sec
Data:   10000 points, Queries:   20 => Build: 2.89 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 25.96 sec, Search: 0.16 sec
Data:   10000 points, Queries:   20 => Build: 3.19 sec, Search: 0.04 sec
Data:   10000 points, Queries:  500 => Build: 2.49 sec, Search: 0.80 sec
Data:  100000 points, Queries:   20 => Build

**-----VP_MWV Run 2-------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 698.46it/s]


Build Time: 1.48 sec, Search Time: 0.14 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 875.99it/s]


Build Time: 2.54 sec, Search Time: 0.11 sec

3: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 23.61 sec, Search Time: 0.04 sec

14: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 595.26it/s]


Build Time: 24.25 sec, Search Time: 0.83 sec

15: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 588.89it/s]

Build Time: 22.51 sec, Search Time: 0.17 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 1.48 sec, Search: 0.14 sec
Data:   10000 points, Queries:  100 => Build: 2.54 sec, Search: 0.11 sec
Data:   10000 points, Queries:   20 => Build: 2.18 sec, Search: 0.03 sec
Data: 1000000 points, Queries:   20 => Build: 245.37 sec, Search: 0.11 sec
Data:   10000 points, Queries:  100 => Build: 1.45 sec, Search: 0.16 sec
Data:   10000 points, Queries:  100 => Build: 3.13 sec, Search: 0.22 sec
Data:   10000 points, Queries:  500 => Build: 2.10 sec, Search: 0.79 sec
Data:  100000 points, Queries:   20 => Build: 29.85 sec, Search: 0.04 sec
Data:   10000 points, Queries:   20 => Build: 3.22 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 21.60 sec, Search: 0.18 sec
Data:   10000 points, Queries:   20 => Build: 2.83 sec, Search: 0.04 sec
Data:   10000 points, Queries:  500 => Build: 2.95 sec, Search: 0.77 sec
Data:  100000 points, Queries:   20 => Build

**-------VP_MWV Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 563.31it/s]


Build Time: 2.50 sec, Search Time: 0.18 sec

2: === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 650.27it/s]


Build Time: 1.51 sec, Search Time: 0.15 sec

3: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 24.64 sec, Search Time: 0.03 sec

14: === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 585.62it/s]


Build Time: 28.37 sec, Search Time: 0.84 sec

15: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 449.62it/s]

Build Time: 27.69 sec, Search Time: 0.22 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 2.50 sec, Search: 0.18 sec
Data:   10000 points, Queries:  100 => Build: 1.51 sec, Search: 0.15 sec
Data:   10000 points, Queries:   20 => Build: 2.60 sec, Search: 0.03 sec
Data: 1000000 points, Queries:   20 => Build: 244.97 sec, Search: 0.06 sec
Data:   10000 points, Queries:  100 => Build: 2.61 sec, Search: 0.14 sec
Data:   10000 points, Queries:  100 => Build: 4.18 sec, Search: 0.12 sec
Data:   10000 points, Queries:  500 => Build: 3.68 sec, Search: 0.64 sec
Data:  100000 points, Queries:   20 => Build: 24.80 sec, Search: 0.04 sec
Data:   10000 points, Queries:   20 => Build: 2.45 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Build: 26.88 sec, Search: 0.17 sec
Data:   10000 points, Queries:   20 => Build: 4.11 sec, Search: 0.03 sec
Data:   10000 points, Queries:  500 => Build: 1.50 sec, Search: 0.79 sec
Data:  100000 points, Queries:   20 => Build

**-------VP_MWV Analysis------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 394.63',
    'Total Time: 392.71',
    'Total Time: 405.95'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 397.7633 sec, Std Dev: 7.1546 sec, CV: 1.80%


--------------

**------------------Ball Tree--------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=40):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
    #*-----------------------------------------------------------------------------

    def insert(self, point):
        """
        Insert a new point into the Ball Tree.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        # Path to keep track of nodes to update
        path = []

        # Traverse the tree to find the appropriate leaf
        current = self.root
        path.append(current)

        while current.points is None:  # Continue until we reach a leaf node
            # Calculate distances to pivots
            dist_to_p1 = np.sqrt(np.sum((point - current.pivot1) ** 2))
            dist_to_p2 = np.sqrt(np.sum((point - current.pivot2) ** 2))

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid)
        # New centroid = (n * old_centroid + new_point) / (n + 1)
        current.center = (n * current.center + point) / (n + 1)

        # Update the radius if necessary
        dist_to_center = np.sqrt(np.sum((point - current.center) ** 2))
        current.radius = max(current.radius, dist_to_center)

        # Update centers and radii of ancestor nodes
        path.pop()  # Remove the leaf node as we've already updated it
        self._update_ancestors(path, point)

    #*-----------------------------------------------------------------------------
    def _update_ancestors(self, path, point):
        """
        Update the center and radius of all nodes in the path after insertion.

        Parameters:
            path (list): List of nodes from root to leaf (excluding the leaf)
            point (np.ndarray): The point that was inserted
        """
        # Process nodes from bottom to top (leaf's parent first)
        for node in reversed(path):
            # For center, we would ideally recompute based on all points
            # but as an approximation, we'll slightly shift it toward the new point
            # This way we avoid having to recursively gather all points

            # We'll apply a small weight to the new point based on estimated subtree size
            # Assume a balanced tree with approximately 2^depth nodes at the bottom
            depth = len(path) - path.index(node)
            weight = 1.0 / (2**depth)

            # Update center with minimal shift toward new point
            node.center = (1 - weight) * node.center + weight * point

            # Check if radius needs update by calculating distance from new point to center
            dist_to_center = np.sqrt(np.sum((point - node.center) ** 2))

            # Only update radius if the new point is outside the current ball
            if dist_to_center > node.radius:
                node.radius = dist_to_center

    #*-----------------------------------------------------------------------------
    def insert_opt(self, point):
        """
        Insert a new point into the Ball Tree with optimized performance.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        current = self.root
        nodes_to_update = []  # Store nodes that need radius updates

        # Traverse the tree to find the appropriate leaf
        while current.points is None:  # Continue until we reach a leaf node
            # Check if point is within current node's radius
            dist_to_center = np.linalg.norm(point - current.center)
            if dist_to_center > current.radius:
                nodes_to_update.append((current, dist_to_center))

            # Calculate distances to pivots (only once per level)
            dist_to_p1 = np.linalg.norm(point - current.pivot1)
            dist_to_p2 = np.linalg.norm(point - current.pivot2)

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid) - avoid creating temporary arrays
        current.center *= n
        current.center += point
        current.center /= (n + 1)

        # Update the radius if necessary
        dist_to_center = np.linalg.norm(point - current.center)
        if dist_to_center > current.radius:
            current.radius = dist_to_center

        # Update only radii of nodes that need updates
        for node, dist in nodes_to_update:
            node.radius = max(node.radius, dist)
#*-----------------------------------------------------------------------------

---------

**--------Ball Tree (Run 1)----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 2127.98it/s]


Build Time: 0.02 sec, Search Time: 0.05 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 1224.45it/s]

Build Time: 0.02 sec, Search Time: 0.08 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 4.18 sec, Search Time: 0.20 sec

14:  === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:06<00:00, 81.25it/s]


Build Time: 4.52 sec, Search Time: 6.11 sec

15:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 76.66it/s]

Build Time: 4.85 sec, Search Time: 1.29 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 0.02 sec, Search: 0.05 sec
Data:   10000 points, Queries:  100 => Build: 0.02 sec, Search: 0.08 sec
Data:   10000 points, Queries:   20 => Build: 0.04 sec, Search: 0.02 sec
Data: 1000000 points, Queries:   20 => Build: 3.28 sec, Search: 0.22 sec
Data:   10000 points, Queries:  100 => Build: 3.06 sec, Search: 1.19 sec
Data:   10000 points, Queries:  100 => Build: 2.99 sec, Search: 1.04 sec
Data:   10000 points, Queries:  500 => Build: 3.04 sec, Search: 5.63 sec
Data:  100000 points, Queries:   20 => Build: 3.38 sec, Search: 0.22 sec
Data:   10000 points, Queries:   20 => Build: 3.38 sec, Search: 0.22 sec
Data:  100000 points, Queries:  100 => Build: 3.72 sec, Search: 1.10 sec
Data:   10000 points, Queries:   20 => Build: 3.72 sec, Search: 0.24 sec
Data:   10000 points, Queries:  500 => Build: 3.76 sec, Search: 6.07 sec
Data:  100000 points, Queries:   20 => Build: 4.1

**-------Ball Run 2----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 2090.34it/s]


Build Time: 0.01 sec, Search Time: 0.05 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 1173.06it/s]


Build Time: 0.02 sec, Search Time: 0.08 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 4.02 sec, Search Time: 0.22 sec

14:  === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:05<00:00, 84.86it/s]


Build Time: 4.52 sec, Search Time: 5.84 sec

15:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 80.88it/s]

Build Time: 5.05 sec, Search Time: 1.23 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 0.01 sec, Search: 0.05 sec
Data:   10000 points, Queries:  100 => Build: 0.02 sec, Search: 0.08 sec
Data:   10000 points, Queries:   20 => Build: 0.03 sec, Search: 0.03 sec
Data: 1000000 points, Queries:   20 => Build: 2.80 sec, Search: 0.24 sec
Data:   10000 points, Queries:  100 => Build: 2.86 sec, Search: 1.02 sec
Data:   10000 points, Queries:  100 => Build: 2.89 sec, Search: 1.02 sec
Data:   10000 points, Queries:  500 => Build: 2.94 sec, Search: 5.56 sec
Data:  100000 points, Queries:   20 => Build: 3.23 sec, Search: 0.21 sec
Data:   10000 points, Queries:   20 => Build: 3.28 sec, Search: 0.22 sec
Data:  100000 points, Queries:  100 => Build: 3.74 sec, Search: 1.17 sec
Data:   10000 points, Queries:   20 => Build: 3.73 sec, Search: 0.23 sec
Data:   10000 points, Queries:  500 => Build: 3.84 sec, Search: 5.79 sec
Data:  100000 points, Queries:   20 => Build: 4.0

**---------Ball Tree Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 2287.47it/s]


Build Time: 0.01 sec, Search Time: 0.04 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 1109.92it/s]


Build Time: 0.02 sec, Search Time: 0.09 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 4.10 sec, Search Time: 0.19 sec

14:  === Testing with Data Size: 100,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:06<00:00, 77.84it/s]


Build Time: 4.45 sec, Search Time: 6.37 sec

15:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 76.87it/s]

Build Time: 4.78 sec, Search Time: 1.29 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:  100 => Build: 0.01 sec, Search: 0.04 sec
Data:   10000 points, Queries:  100 => Build: 0.02 sec, Search: 0.09 sec
Data:   10000 points, Queries:   20 => Build: 0.03 sec, Search: 0.02 sec
Data: 1000000 points, Queries:   20 => Build: 2.86 sec, Search: 0.20 sec
Data:   10000 points, Queries:  100 => Build: 2.92 sec, Search: 1.15 sec
Data:   10000 points, Queries:  100 => Build: 2.98 sec, Search: 1.00 sec
Data:   10000 points, Queries:  500 => Build: 2.99 sec, Search: 5.37 sec
Data:  100000 points, Queries:   20 => Build: 3.30 sec, Search: 0.24 sec
Data:   10000 points, Queries:   20 => Build: 3.33 sec, Search: 0.22 sec
Data:  100000 points, Queries:  100 => Build: 3.66 sec, Search: 1.13 sec
Data:   10000 points, Queries:   20 => Build: 3.73 sec, Search: 0.25 sec
Data:   10000 points, Queries:  500 => Build: 3.69 sec, Search: 5.97 sec
Data:  100000 points, Queries:   20 => Build: 4.1

**-------Ball Tree Analysis-----------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 67.62',
    'Total Time: 65.88',
    'Total Time: 66.40'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 66.6333 sec, Std Dev: 0.8932 sec, CV: 1.34%


--------------------

**----------QuickSelect Algorithm-----------**

In [10]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

-----------------------------------

**--------KD Hybrid---------------**

In [11]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

---------------------------

**---------KD Hybrid Sqrt Run 1---------**

In [12]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0196 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2583.10it/s]


Total Search Time: 0.046985 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 151478.52it/s]


Insert Time: 0.0681 sec


...

Total Search Time: 15.575246 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 121480.12it/s]


Insert Time: 0.8257 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 26.33it/s]

Total Search Time: 4.282202 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0196 sec, Total Search Time: 0.046985 sec, Total Time: 0.0666 sec
Points: 20,000, Insert Time: 0.0681 sec, Total Search Time: 0.072761 sec, Total Time: 0.1409 sec
Points: 30,000, Build Time: 0.1246 sec, Total Search Time: 0.028075 sec, Total Time: 0.1527 sec
Points: 1,030,000, Build Time: 5.1768 sec, Total Search Time: 0.906612 sec, Total Time: 6.0834 sec
Points: 1,040,000, Insert Time: 0.0929 sec, Total Search Time: 2.860462 sec, Total Time: 2.9534 sec
Points: 1,050,000, Insert Time: 0.1338 sec, Total Search Time: 3.510327 sec, Total Time: 3.6441 sec
Points: 1,060,000, Insert Time: 0.0822 sec, Total Search Time: 15.793580 sec, Total Time: 15.8757 sec
Points: 1,160,000, Insert Time: 0.7961 sec, Total Search Time: 0.798970 sec, Total Time: 1.5951 sec
Points: 1,170,000, Insert Time: 0.0955 sec, Total Search Time: 0.933579 sec, Total Time: 1.0291 sec
Points: 1,270,000, Insert Time: 0.922

**---------KD Hybrid Sqrt Run 2----------**

In [17]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0982 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2768.54it/s]


Total Search Time: 0.041322 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 116709.44it/s]


Insert Time: 0.0876 sec


...

Total Search Time: 15.553519 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 112716.35it/s]


Insert Time: 0.8894 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 27.78it/s]

Total Search Time: 4.106149 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0982 sec, Total Search Time: 0.041322 sec, Total Time: 0.1396 sec
Points: 20,000, Insert Time: 0.0876 sec, Total Search Time: 0.073115 sec, Total Time: 0.1608 sec
Points: 30,000, Build Time: 0.0825 sec, Total Search Time: 0.028372 sec, Total Time: 0.1109 sec
Points: 1,030,000, Build Time: 5.3729 sec, Total Search Time: 0.726684 sec, Total Time: 6.0996 sec
Points: 1,040,000, Insert Time: 0.0921 sec, Total Search Time: 2.835216 sec, Total Time: 2.9273 sec
Points: 1,050,000, Insert Time: 0.0888 sec, Total Search Time: 2.828393 sec, Total Time: 2.9172 sec
Points: 1,060,000, Insert Time: 0.1040 sec, Total Search Time: 13.353529 sec, Total Time: 13.4575 sec
Points: 1,160,000, Insert Time: 0.9356 sec, Total Search Time: 0.791249 sec, Total Time: 1.7269 sec
Points: 1,170,000, Insert Time: 0.0776 sec, Total Search Time: 0.812622 sec, Total Time: 0.8902 sec
Points: 1,270,000, Insert Time: 0.922

**------KD Hybrid Sqrt Run 3--------**

In [22]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0196 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2757.07it/s]


Total Search Time: 0.041530 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 149703.01it/s]


Insert Time: 0.0693 sec


...

Total Search Time: 15.348184 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 113874.87it/s]


Insert Time: 0.8801 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 32.81it/s]

Total Search Time: 3.525063 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0196 sec, Total Search Time: 0.041530 sec, Total Time: 0.0612 sec
Points: 20,000, Insert Time: 0.0693 sec, Total Search Time: 0.071282 sec, Total Time: 0.1406 sec
Points: 30,000, Build Time: 0.0862 sec, Total Search Time: 0.028999 sec, Total Time: 0.1152 sec
Points: 1,030,000, Build Time: 5.4822 sec, Total Search Time: 0.717188 sec, Total Time: 6.1994 sec
Points: 1,040,000, Insert Time: 0.1013 sec, Total Search Time: 2.789199 sec, Total Time: 2.8905 sec
Points: 1,050,000, Insert Time: 0.0764 sec, Total Search Time: 2.817962 sec, Total Time: 2.8943 sec
Points: 1,060,000, Insert Time: 0.0811 sec, Total Search Time: 13.380136 sec, Total Time: 13.4612 sec
Points: 1,160,000, Insert Time: 0.9376 sec, Total Search Time: 0.780972 sec, Total Time: 1.7185 sec
Points: 1,170,000, Insert Time: 0.0870 sec, Total Search Time: 0.811128 sec, Total Time: 0.8981 sec
Points: 1,270,000, Insert Time: 0.888

**----------KD Hybrid Sqrt Analysis-------------**

In [27]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 78.0033',
    'Total Time:: 71.5003',
    'Total Time:: 70.3410'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 73.2815 sec, Std Dev: 4.1300 sec, CV: 5.64%


--------------------------------

**------KD Hybrid Log Run 1--------**

In [13]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0221 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2695.45it/s]


Total Search Time: 0.042140 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 148786.42it/s]


Insert Time: 0.0693 sec


...

Total Search Time: 18.174513 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 114290.88it/s]


Insert Time: 0.8771 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 31.80it/s]

Total Search Time: 3.625902 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0221 sec, Total Search Time: 0.042140 sec, Total Time: 0.0642 sec
Points: 20,000, Insert Time: 0.0693 sec, Total Search Time: 0.071026 sec, Total Time: 0.1403 sec
Points: 30,000, Insert Time: 0.0716 sec, Total Search Time: 0.028120 sec, Total Time: 0.0997 sec
Points: 1,030,000, Build Time: 5.4349 sec, Total Search Time: 0.861641 sec, Total Time: 6.2966 sec
Points: 1,040,000, Insert Time: 0.0787 sec, Total Search Time: 2.854020 sec, Total Time: 2.9328 sec
Points: 1,050,000, Insert Time: 0.0923 sec, Total Search Time: 3.343548 sec, Total Time: 3.4358 sec
Points: 1,060,000, Insert Time: 0.0793 sec, Total Search Time: 13.580615 sec, Total Time: 13.6599 sec
Points: 1,160,000, Insert Time: 0.9192 sec, Total Search Time: 0.887816 sec, Total Time: 1.8070 sec
Points: 1,170,000, Insert Time: 0.0866 sec, Total Search Time: 0.809616 sec, Total Time: 0.8962 sec
Points: 1,270,000, Insert Time: 0.83

**-------KD Hybrid Log Run 2-----------**

In [18]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0228 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2724.25it/s]


Total Search Time: 0.041589 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 129585.41it/s]


Insert Time: 0.0790 sec


...

Total Search Time: 18.107602 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 122970.33it/s]


Insert Time: 0.8154 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 31.87it/s]

Total Search Time: 3.599812 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0228 sec, Total Search Time: 0.041589 sec, Total Time: 0.0644 sec
Points: 20,000, Insert Time: 0.0790 sec, Total Search Time: 0.071626 sec, Total Time: 0.1507 sec
Points: 30,000, Insert Time: 0.0747 sec, Total Search Time: 0.030569 sec, Total Time: 0.1053 sec
Points: 1,030,000, Build Time: 5.4118 sec, Total Search Time: 0.807948 sec, Total Time: 6.2198 sec
Points: 1,040,000, Insert Time: 0.1013 sec, Total Search Time: 2.804010 sec, Total Time: 2.9054 sec
Points: 1,050,000, Insert Time: 0.0802 sec, Total Search Time: 3.301557 sec, Total Time: 3.3818 sec
Points: 1,060,000, Insert Time: 0.0801 sec, Total Search Time: 13.289349 sec, Total Time: 13.3694 sec
Points: 1,160,000, Insert Time: 0.8170 sec, Total Search Time: 0.898878 sec, Total Time: 1.7159 sec
Points: 1,170,000, Insert Time: 0.0828 sec, Total Search Time: 0.821178 sec, Total Time: 0.9040 sec
Points: 1,270,000, Insert Time: 0.89

**------KD Hybrid Log Run 3------**

In [23]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0214 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2743.38it/s]


Total Search Time: 0.041358 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 112982.75it/s]


Insert Time: 0.0902 sec


...

Total Search Time: 15.556176 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 104549.88it/s]


Insert Time: 0.9584 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 32.73it/s]

Total Search Time: 3.538988 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0214 sec, Total Search Time: 0.041358 sec, Total Time: 0.0628 sec
Points: 20,000, Insert Time: 0.0902 sec, Total Search Time: 0.069268 sec, Total Time: 0.1595 sec
Points: 30,000, Insert Time: 0.0978 sec, Total Search Time: 0.028408 sec, Total Time: 0.1262 sec
Points: 1,030,000, Build Time: 5.4932 sec, Total Search Time: 0.711937 sec, Total Time: 6.2052 sec
Points: 1,040,000, Insert Time: 0.1377 sec, Total Search Time: 2.783237 sec, Total Time: 2.9209 sec
Points: 1,050,000, Insert Time: 0.0783 sec, Total Search Time: 2.809991 sec, Total Time: 2.8883 sec
Points: 1,060,000, Insert Time: 0.0986 sec, Total Search Time: 13.234489 sec, Total Time: 13.3331 sec
Points: 1,160,000, Insert Time: 0.9073 sec, Total Search Time: 0.792853 sec, Total Time: 1.7001 sec
Points: 1,170,000, Insert Time: 0.0842 sec, Total Search Time: 0.797787 sec, Total Time: 0.8820 sec
Points: 1,270,000, Insert Time: 0.97

**---------KD Hybrid Log Analysis---------**

In [26]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 77.3747',
    'Total Time:: 76.5356 ',
    'Total Time:: 70.6165'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 74.8423 sec, Std Dev: 3.6836 sec, CV: 4.92%


---------------------

**--------KD Hybrid Log Ratio Run 1---------**

In [14]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0230 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2763.32it/s]


Total Search Time: 0.041483 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 148432.58it/s]


Insert Time: 0.0693 sec


...

Total Search Time: 15.579092 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 118629.27it/s]


Insert Time: 0.8451 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 27.66it/s]

Total Search Time: 4.103510 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0230 sec, Total Search Time: 0.041483 sec, Total Time: 0.0645 sec
Points: 20,000, Insert Time: 0.0693 sec, Total Search Time: 0.072976 sec, Total Time: 0.1422 sec
Points: 30,000, Insert Time: 0.0868 sec, Total Search Time: 0.034357 sec, Total Time: 0.1211 sec
Points: 1,030,000, Build Time: 5.4354 sec, Total Search Time: 0.758516 sec, Total Time: 6.1939 sec
Points: 1,040,000, Insert Time: 0.0954 sec, Total Search Time: 2.855805 sec, Total Time: 2.9512 sec
Points: 1,050,000, Insert Time: 0.1086 sec, Total Search Time: 2.866179 sec, Total Time: 2.9748 sec
Points: 1,060,000, Insert Time: 0.0843 sec, Total Search Time: 13.555495 sec, Total Time: 13.6398 sec
Points: 1,160,000, Insert Time: 0.8852 sec, Total Search Time: 0.788283 sec, Total Time: 1.6735 sec
Points: 1,170,000, Insert Time: 0.0798 sec, Total Search Time: 0.808648 sec, Total Time: 0.8885 sec
Points: 1,270,000, Insert Time: 0.91

**------KD Hybrid Log Ratio Run 2---------**

In [19]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0217 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2826.83it/s]


Total Search Time: 0.040671 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 149174.83it/s]


Insert Time: 0.0692 sec


...

Total Search Time: 18.043558 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 119146.58it/s]


Insert Time: 0.8410 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 31.91it/s]

Total Search Time: 3.606079 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0217 sec, Total Search Time: 0.040671 sec, Total Time: 0.0624 sec
Points: 20,000, Insert Time: 0.0692 sec, Total Search Time: 0.069184 sec, Total Time: 0.1384 sec
Points: 30,000, Insert Time: 0.0674 sec, Total Search Time: 0.029881 sec, Total Time: 0.0973 sec
Points: 1,030,000, Build Time: 5.5022 sec, Total Search Time: 0.817246 sec, Total Time: 6.3194 sec
Points: 1,040,000, Insert Time: 0.0766 sec, Total Search Time: 2.837843 sec, Total Time: 2.9145 sec
Points: 1,050,000, Insert Time: 0.1081 sec, Total Search Time: 3.326614 sec, Total Time: 3.4347 sec
Points: 1,060,000, Insert Time: 0.0975 sec, Total Search Time: 13.480788 sec, Total Time: 13.5783 sec
Points: 1,160,000, Insert Time: 0.9308 sec, Total Search Time: 0.910471 sec, Total Time: 1.8412 sec
Points: 1,170,000, Insert Time: 0.1064 sec, Total Search Time: 0.828160 sec, Total Time: 0.9346 sec
Points: 1,270,000, Insert Time: 0.89

**------KD Hybrid Log Ratio Run 3-----**

In [24]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0253 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2735.40it/s]


Total Search Time: 0.041564 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 107034.41it/s]


Insert Time: 0.0955 sec


...

Total Search Time: 15.329623 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 121734.65it/s]


Insert Time: 0.8234 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 32.60it/s]

Total Search Time: 3.561388 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0253 sec, Total Search Time: 0.041564 sec, Total Time: 0.0669 sec
Points: 20,000, Insert Time: 0.0955 sec, Total Search Time: 0.075458 sec, Total Time: 0.1709 sec
Points: 30,000, Insert Time: 0.0725 sec, Total Search Time: 0.032774 sec, Total Time: 0.1053 sec
Points: 1,030,000, Build Time: 5.4460 sec, Total Search Time: 0.708003 sec, Total Time: 6.1540 sec
Points: 1,040,000, Insert Time: 0.0808 sec, Total Search Time: 2.806774 sec, Total Time: 2.8876 sec
Points: 1,050,000, Insert Time: 0.0763 sec, Total Search Time: 2.810125 sec, Total Time: 2.8865 sec
Points: 1,060,000, Insert Time: 0.0772 sec, Total Search Time: 13.267459 sec, Total Time: 13.3446 sec
Points: 1,160,000, Insert Time: 0.7816 sec, Total Search Time: 0.767489 sec, Total Time: 1.5491 sec
Points: 1,170,000, Insert Time: 0.1255 sec, Total Search Time: 0.794275 sec, Total Time: 0.9197 sec
Points: 1,270,000, Insert Time: 0.92

**---------KD Hybrid Log Ratio Analysis----------**

In [25]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 71.4131',
    'Total Time:: 77.2205',
    'Total Time:: 69.9392'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 72.8576 sec, Std Dev: 3.8496 sec, CV: 5.28%


------------------------

**----------KD Tree Epsilon Dynamic---------**

In [28]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

-----------------

**-------kd dyn sqrt run 1------------**

In [29]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0206 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2827.82it/s]


Total Search Time: 0.040719 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 141668.83it/s]


Insert Time: 0.0728 sec


...

Total Search Time: 10.640275 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 93560.06it/s]


Insert Time: 1.0704 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 48.50it/s]

Total Search Time: 2.602042 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0206 sec, Total Search Time: 0.040719 sec, Total Time: 0.0613 sec
Points: 20,000, Insert Time: 0.0728 sec, Total Search Time: 0.066891 sec, Total Time: 0.1397 sec
Points: 30,000, Build Time: 0.1004 sec, Total Search Time: 0.026984 sec, Total Time: 0.1274 sec
Points: 1,030,000, Build Time: 7.6732 sec, Total Search Time: 0.626282 sec, Total Time: 8.2995 sec
Points: 1,040,000, Insert Time: 0.1120 sec, Total Search Time: 2.125302 sec, Total Time: 2.2373 sec
Points: 1,050,000, Insert Time: 0.1289 sec, Total Search Time: 2.146176 sec, Total Time: 2.2751 sec
Points: 1,060,000, Insert Time: 0.1139 sec, Total Search Time: 9.514876 sec, Total Time: 9.6288 sec
Points: 1,160,000, Insert Time: 1.1150 sec, Total Search Time: 0.702983 sec, Total Time: 1.8180 sec
Points: 1,170,000, Insert Time: 0.1106 sec, Total Search Time: 0.701297 sec, Total Time: 0.8119 sec
Points: 1,270,000, Insert Time: 1.1006 

**---------kd dyn sqrt run 2-----------**

In [35]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0199 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2783.68it/s]


Total Search Time: 0.041473 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 148477.77it/s]


Insert Time: 0.0696 sec


...

Total Search Time: 10.701597 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 91675.11it/s]


Insert Time: 1.0937 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 48.08it/s]

Total Search Time: 2.631994 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0199 sec, Total Search Time: 0.041473 sec, Total Time: 0.0614 sec
Points: 20,000, Insert Time: 0.0696 sec, Total Search Time: 0.069791 sec, Total Time: 0.1394 sec
Points: 30,000, Build Time: 0.0975 sec, Total Search Time: 0.026173 sec, Total Time: 0.1237 sec
Points: 1,030,000, Build Time: 8.0230 sec, Total Search Time: 0.637586 sec, Total Time: 8.6606 sec
Points: 1,040,000, Insert Time: 0.1546 sec, Total Search Time: 2.130490 sec, Total Time: 2.2851 sec
Points: 1,050,000, Insert Time: 0.0927 sec, Total Search Time: 2.136719 sec, Total Time: 2.2294 sec
Points: 1,060,000, Insert Time: 0.1070 sec, Total Search Time: 9.609613 sec, Total Time: 9.7166 sec
Points: 1,160,000, Insert Time: 1.0966 sec, Total Search Time: 0.701942 sec, Total Time: 1.7986 sec
Points: 1,170,000, Insert Time: 0.1117 sec, Total Search Time: 0.713044 sec, Total Time: 0.8248 sec
Points: 1,270,000, Insert Time: 1.0713 

**---------KD dyn sqrt run 3-----------**

In [40]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0209 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2722.44it/s]


Total Search Time: 0.042401 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 126488.38it/s]


Insert Time: 0.0813 sec


...

Total Search Time: 10.681046 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 96697.02it/s]


Insert Time: 1.0360 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 47.69it/s]

Total Search Time: 2.620475 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0209 sec, Total Search Time: 0.042401 sec, Total Time: 0.0633 sec
Points: 20,000, Insert Time: 0.0813 sec, Total Search Time: 0.069640 sec, Total Time: 0.1509 sec
Points: 30,000, Build Time: 0.1055 sec, Total Search Time: 0.026485 sec, Total Time: 0.1320 sec
Points: 1,030,000, Build Time: 8.0027 sec, Total Search Time: 0.629602 sec, Total Time: 8.6323 sec
Points: 1,040,000, Insert Time: 0.1175 sec, Total Search Time: 2.119691 sec, Total Time: 2.2372 sec
Points: 1,050,000, Insert Time: 0.1164 sec, Total Search Time: 2.139716 sec, Total Time: 2.2562 sec
Points: 1,060,000, Insert Time: 0.1190 sec, Total Search Time: 9.594329 sec, Total Time: 9.7133 sec
Points: 1,160,000, Insert Time: 0.9897 sec, Total Search Time: 0.710884 sec, Total Time: 1.7006 sec
Points: 1,170,000, Insert Time: 0.1352 sec, Total Search Time: 0.716363 sec, Total Time: 0.8516 sec
Points: 1,270,000, Insert Time: 1.1800 

**---------kd dyn sqrt analysis------------**

In [45]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 57.1119',
    'Total Time:: 57.7352',
    'Total Time:: 57.7748'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.5406 sec, Std Dev: 0.3718 sec, CV: 0.65%


--------------------

**-------KD  Dyn Log Threshold Run 1-----**

In [30]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.1319 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2872.30it/s]


Total Search Time: 0.040049 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 152589.50it/s]


Insert Time: 0.0673 sec


...

Total Search Time: 10.713140 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 89818.89it/s]


Insert Time: 1.1155 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 47.66it/s]

Total Search Time: 2.661743 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.1319 sec, Total Search Time: 0.040049 sec, Total Time: 0.1719 sec
Points: 20,000, Insert Time: 0.0673 sec, Total Search Time: 0.068468 sec, Total Time: 0.1358 sec
Points: 30,000, Insert Time: 0.0801 sec, Total Search Time: 0.030079 sec, Total Time: 0.1102 sec
Points: 1,030,000, Build Time: 7.7562 sec, Total Search Time: 0.628716 sec, Total Time: 8.3849 sec
Points: 1,040,000, Insert Time: 0.1316 sec, Total Search Time: 2.144182 sec, Total Time: 2.2758 sec
Points: 1,050,000, Insert Time: 0.0963 sec, Total Search Time: 2.129326 sec, Total Time: 2.2256 sec
Points: 1,060,000, Insert Time: 0.0997 sec, Total Search Time: 9.612244 sec, Total Time: 9.7120 sec
Points: 1,160,000, Insert Time: 1.1032 sec, Total Search Time: 0.711552 sec, Total Time: 1.8147 sec
Points: 1,170,000, Insert Time: 0.1007 sec, Total Search Time: 0.710138 sec, Total Time: 0.8109 sec
Points: 1,270,000, Insert Time: 1.0784

**-----KD Dyn Log Run 2--------**

In [36]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0223 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2838.00it/s]


Total Search Time: 0.040497 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 126455.20it/s]


Insert Time: 0.0811 sec


...

Total Search Time: 10.805457 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 95546.24it/s]


Insert Time: 1.0484 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 47.90it/s]

Total Search Time: 2.639182 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0223 sec, Total Search Time: 0.040497 sec, Total Time: 0.0627 sec
Points: 20,000, Insert Time: 0.0811 sec, Total Search Time: 0.070200 sec, Total Time: 0.1513 sec
Points: 30,000, Insert Time: 0.0781 sec, Total Search Time: 0.028152 sec, Total Time: 0.1062 sec
Points: 1,030,000, Build Time: 7.9851 sec, Total Search Time: 0.648989 sec, Total Time: 8.6341 sec
Points: 1,040,000, Insert Time: 0.1167 sec, Total Search Time: 2.151532 sec, Total Time: 2.2682 sec
Points: 1,050,000, Insert Time: 0.1050 sec, Total Search Time: 2.158830 sec, Total Time: 2.2638 sec
Points: 1,060,000, Insert Time: 0.1224 sec, Total Search Time: 9.563537 sec, Total Time: 9.6860 sec
Points: 1,160,000, Insert Time: 1.1808 sec, Total Search Time: 0.712529 sec, Total Time: 1.8933 sec
Points: 1,170,000, Insert Time: 0.1182 sec, Total Search Time: 0.710506 sec, Total Time: 0.8287 sec
Points: 1,270,000, Insert Time: 1.0286

**----KD Dyn Log Run 3------**

In [41]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0203 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2980.69it/s]


Total Search Time: 0.038940 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 144248.66it/s]


Insert Time: 0.0712 sec


...

Total Search Time: 10.668551 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 90169.12it/s]


Insert Time: 1.1107 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 47.45it/s]

Total Search Time: 2.665508 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0203 sec, Total Search Time: 0.038940 sec, Total Time: 0.0593 sec
Points: 20,000, Insert Time: 0.0712 sec, Total Search Time: 0.067639 sec, Total Time: 0.1389 sec
Points: 30,000, Insert Time: 0.0781 sec, Total Search Time: 0.029421 sec, Total Time: 0.1075 sec
Points: 1,030,000, Build Time: 7.9820 sec, Total Search Time: 0.630593 sec, Total Time: 8.6126 sec
Points: 1,040,000, Insert Time: 0.0959 sec, Total Search Time: 2.162480 sec, Total Time: 2.2584 sec
Points: 1,050,000, Insert Time: 0.1042 sec, Total Search Time: 2.116471 sec, Total Time: 2.2207 sec
Points: 1,060,000, Insert Time: 0.0940 sec, Total Search Time: 9.586723 sec, Total Time: 9.6807 sec
Points: 1,160,000, Insert Time: 0.9790 sec, Total Search Time: 0.701394 sec, Total Time: 1.6803 sec
Points: 1,170,000, Insert Time: 0.1043 sec, Total Search Time: 0.702844 sec, Total Time: 0.8071 sec
Points: 1,270,000, Insert Time: 1.0685

**--------kd dyn log analysis--------**

In [44]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 57.7876',
    'Total Time:: 57.8423',
    'Total Time:: 57.4435'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.6911 sec, Std Dev: 0.2162 sec, CV: 0.37%


-------------------

**------KD Tree Dynamic Log Ratio Run 1--------**

In [31]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0200 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2730.74it/s]


Total Search Time: 0.042003 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 116835.91it/s]


Insert Time: 0.0874 sec


...

Total Search Time: 10.694446 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 88775.85it/s]


Insert Time: 1.1283 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 48.51it/s]

Total Search Time: 2.579512 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0200 sec, Total Search Time: 0.042003 sec, Total Time: 0.0620 sec
Points: 20,000, Insert Time: 0.0874 sec, Total Search Time: 0.068085 sec, Total Time: 0.1554 sec
Points: 30,000, Insert Time: 0.0684 sec, Total Search Time: 0.030383 sec, Total Time: 0.0988 sec
Points: 1,030,000, Build Time: 7.9858 sec, Total Search Time: 0.640413 sec, Total Time: 8.6262 sec
Points: 1,040,000, Insert Time: 0.1045 sec, Total Search Time: 2.128263 sec, Total Time: 2.2328 sec
Points: 1,050,000, Insert Time: 0.0936 sec, Total Search Time: 2.153579 sec, Total Time: 2.2472 sec
Points: 1,060,000, Insert Time: 0.1015 sec, Total Search Time: 9.710921 sec, Total Time: 9.8125 sec
Points: 1,160,000, Insert Time: 1.1341 sec, Total Search Time: 0.702409 sec, Total Time: 1.8365 sec
Points: 1,170,000, Insert Time: 0.1076 sec, Total Search Time: 0.697957 sec, Total Time: 0.8055 sec
Points: 1,270,000, Insert Time: 1.0784

**--------KD Dyn Log Ratio Run 2------**

In [37]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0212 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2961.43it/s]


Total Search Time: 0.038834 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123005.17it/s]


Insert Time: 0.0832 sec


...

Total Search Time: 10.748127 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 101939.01it/s]


Insert Time: 0.9829 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 47.49it/s]

Total Search Time: 2.635000 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0212 sec, Total Search Time: 0.038834 sec, Total Time: 0.0601 sec
Points: 20,000, Insert Time: 0.0832 sec, Total Search Time: 0.071415 sec, Total Time: 0.1546 sec
Points: 30,000, Insert Time: 0.0800 sec, Total Search Time: 0.030868 sec, Total Time: 0.1109 sec
Points: 1,030,000, Build Time: 7.9962 sec, Total Search Time: 0.625202 sec, Total Time: 8.6214 sec
Points: 1,040,000, Insert Time: 0.1711 sec, Total Search Time: 2.132069 sec, Total Time: 2.3031 sec
Points: 1,050,000, Insert Time: 0.1190 sec, Total Search Time: 2.119075 sec, Total Time: 2.2380 sec
Points: 1,060,000, Insert Time: 0.1698 sec, Total Search Time: 9.545567 sec, Total Time: 9.7154 sec
Points: 1,160,000, Insert Time: 0.9597 sec, Total Search Time: 0.702063 sec, Total Time: 1.6618 sec
Points: 1,170,000, Insert Time: 0.1359 sec, Total Search Time: 0.711996 sec, Total Time: 0.8479 sec
Points: 1,270,000, Insert Time: 1.1424

**-------KD Dynamic Log Ratio Run 3----**

In [42]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding KD Tree from scratch with 10,000 points.
Build Time: 0.0224 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 2904.34it/s]


Total Search Time: 0.039686 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 145835.07it/s]


Insert Time: 0.0702 sec


...

Total Search Time: 10.702171 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 93176.03it/s]


Insert Time: 1.0752 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 48.04it/s]

Total Search Time: 2.617097 sec

=== Summary of KD Tree Scaling Test ===
Points: 10,000, Build Time: 0.0224 sec, Total Search Time: 0.039686 sec, Total Time: 0.0621 sec
Points: 20,000, Insert Time: 0.0702 sec, Total Search Time: 0.070012 sec, Total Time: 0.1402 sec
Points: 30,000, Insert Time: 0.0923 sec, Total Search Time: 0.029641 sec, Total Time: 0.1219 sec
Points: 1,030,000, Build Time: 8.1073 sec, Total Search Time: 0.633101 sec, Total Time: 8.7404 sec
Points: 1,040,000, Insert Time: 0.1017 sec, Total Search Time: 2.130962 sec, Total Time: 2.2327 sec
Points: 1,050,000, Insert Time: 0.1388 sec, Total Search Time: 2.111761 sec, Total Time: 2.2506 sec
Points: 1,060,000, Insert Time: 0.0983 sec, Total Search Time: 9.517967 sec, Total Time: 9.6162 sec
Points: 1,160,000, Insert Time: 1.0721 sec, Total Search Time: 0.708473 sec, Total Time: 1.7806 sec
Points: 1,170,000, Insert Time: 0.1012 sec, Total Search Time: 0.702881 sec, Total Time: 0.8041 sec
Points: 1,270,000, Insert Time: 1.0523

**----------kd dyn log ratio analysis----------**

In [43]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 57.8627',
    'Total Time:: 57.3866',
    'Total Time:: 57.4435'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 57.5643 sec, Std Dev: 0.2600 sec, CV: 0.45%


--------------------

**----------VP Hybrid------**

In [46]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

------------------------------

**----------vp hybrid sqrt run 1------------**

In [50]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0152 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3742.31it/s]


Total Search Time: 0.030294 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123872.31it/s]


Insert Time: 0.0833 sec


...

Total Search Time: 2.742337 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59384.92it/s]


Insert Time: 1.6863 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 164.47it/s]

Total Search Time: 0.871017 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0152 sec, Total Search Time: 0.030294 sec, Total Time: 0.0455 sec
Points: 20,000, Insert Time: 0.0833 sec, Total Search Time: 0.054198 sec, Total Time: 0.1375 sec
Points: 30,000, Build Time: 0.0553 sec, Total Search Time: 0.017262 sec, Total Time: 0.0725 sec
Points: 1,030,000, Build Time: 4.4783 sec, Total Search Time: 0.007738 sec, Total Time: 4.4861 sec
Points: 1,040,000, Insert Time: 0.1482 sec, Total Search Time: 0.034287 sec, Total Time: 0.1825 sec
Points: 1,050,000, Insert Time: 0.1684 sec, Total Search Time: 0.049798 sec, Total Time: 0.2182 sec
Points: 1,060,000, Insert Time: 0.1705 sec, Total Search Time: 0.247211 sec, Total Time: 0.4177 sec
Points: 1,160,000, Insert Time: 1.6713 sec, Total Search Time: 0.095152 sec, Total Time: 1.7664 sec
Points: 1,170,000, Insert Time: 0.1752 sec, Total Search Time: 0.085401 sec, Total Time: 0.2606 sec
Points: 1,270,000, Insert Time: 1.7190 

**----------vp hybrid sqrt run 2------**

In [55]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0125 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3676.80it/s]


Total Search Time: 0.030585 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 51781.66it/s]


Insert Time: 0.1953 sec


...

Total Search Time: 2.593760 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 60502.44it/s]


Insert Time: 1.6553 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 181.65it/s]

Total Search Time: 0.821980 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0125 sec, Total Search Time: 0.030585 sec, Total Time: 0.0431 sec
Points: 20,000, Insert Time: 0.1953 sec, Total Search Time: 0.052127 sec, Total Time: 0.2474 sec
Points: 30,000, Build Time: 0.0566 sec, Total Search Time: 0.017271 sec, Total Time: 0.0739 sec
Points: 1,030,000, Build Time: 4.2499 sec, Total Search Time: 0.017301 sec, Total Time: 4.2672 sec
Points: 1,040,000, Insert Time: 0.1463 sec, Total Search Time: 0.050774 sec, Total Time: 0.1970 sec
Points: 1,050,000, Insert Time: 0.1657 sec, Total Search Time: 0.063266 sec, Total Time: 0.2289 sec
Points: 1,060,000, Insert Time: 0.1721 sec, Total Search Time: 0.298840 sec, Total Time: 0.4709 sec
Points: 1,160,000, Insert Time: 1.6608 sec, Total Search Time: 0.091846 sec, Total Time: 1.7526 sec
Points: 1,170,000, Insert Time: 0.1675 sec, Total Search Time: 0.090857 sec, Total Time: 0.2583 sec
Points: 1,270,000, Insert Time: 1.6774 

**----------vp hybrid sqrt run 3----------**

In [60]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0148 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 4026.52it/s]


Total Search Time: 0.028413 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 121498.66it/s]


Insert Time: 0.0845 sec


...

Total Search Time: 2.092282 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 56575.82it/s]


Insert Time: 1.7696 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 208.23it/s]

Total Search Time: 0.791276 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0148 sec, Total Search Time: 0.028413 sec, Total Time: 0.0433 sec
Points: 20,000, Insert Time: 0.0845 sec, Total Search Time: 0.160191 sec, Total Time: 0.2447 sec
Points: 30,000, Build Time: 0.0579 sec, Total Search Time: 0.015385 sec, Total Time: 0.0733 sec
Points: 1,030,000, Build Time: 4.8463 sec, Total Search Time: 0.020988 sec, Total Time: 4.8673 sec
Points: 1,040,000, Insert Time: 0.1492 sec, Total Search Time: 0.040148 sec, Total Time: 0.1893 sec
Points: 1,050,000, Insert Time: 0.1690 sec, Total Search Time: 0.054179 sec, Total Time: 0.2232 sec
Points: 1,060,000, Insert Time: 0.1752 sec, Total Search Time: 0.256237 sec, Total Time: 0.4315 sec
Points: 1,160,000, Insert Time: 1.7072 sec, Total Search Time: 0.102509 sec, Total Time: 1.8097 sec
Points: 1,170,000, Insert Time: 0.1720 sec, Total Search Time: 0.101093 sec, Total Time: 0.2731 sec
Points: 1,270,000, Insert Time: 1.6775 

**-------vp hybrid sqrt analysis---------**

In [65]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 20.4957',
    'Total Time:: 20.2803',
    'Total Time:: 20.1722'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 20.3161 sec, Std Dev: 0.1647 sec, CV: 0.81%


----------------------

**-----VP hybrid Log Threshold Run 1------**

In [51]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0116 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 701.40it/s]


Total Search Time: 0.146374 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 112579.72it/s]


Insert Time: 0.0910 sec


...

Total Search Time: 2.719457 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59264.40it/s]


Insert Time: 1.6899 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 171.75it/s]

Total Search Time: 0.831943 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0116 sec, Total Search Time: 0.146374 sec, Total Time: 0.1579 sec
Points: 20,000, Insert Time: 0.0910 sec, Total Search Time: 0.056977 sec, Total Time: 0.1480 sec
Points: 30,000, Insert Time: 0.0875 sec, Total Search Time: 0.022695 sec, Total Time: 0.1102 sec
Points: 1,030,000, Build Time: 4.5291 sec, Total Search Time: 0.007105 sec, Total Time: 4.5362 sec
Points: 1,040,000, Insert Time: 0.1490 sec, Total Search Time: 0.032553 sec, Total Time: 0.1816 sec
Points: 1,050,000, Insert Time: 0.1716 sec, Total Search Time: 0.049540 sec, Total Time: 0.2211 sec
Points: 1,060,000, Insert Time: 0.1774 sec, Total Search Time: 0.242429 sec, Total Time: 0.4198 sec
Points: 1,160,000, Insert Time: 1.7086 sec, Total Search Time: 0.085131 sec, Total Time: 1.7938 sec
Points: 1,170,000, Insert Time: 0.1703 sec, Total Search Time: 0.086021 sec, Total Time: 0.2563 sec
Points: 1,270,000, Insert Time: 1.7026

**-------VP Hybrid log Run 2-----**

In [56]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0144 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3724.17it/s]


Total Search Time: 0.030116 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 48475.73it/s]


Insert Time: 0.2087 sec


...

Total Search Time: 2.600387 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59976.89it/s]


Insert Time: 1.6696 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 184.34it/s]

Total Search Time: 0.810015 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0144 sec, Total Search Time: 0.030116 sec, Total Time: 0.0445 sec
Points: 20,000, Insert Time: 0.2087 sec, Total Search Time: 0.054540 sec, Total Time: 0.2632 sec
Points: 30,000, Insert Time: 0.0858 sec, Total Search Time: 0.024833 sec, Total Time: 0.1107 sec
Points: 1,030,000, Build Time: 4.2231 sec, Total Search Time: 0.018213 sec, Total Time: 4.2413 sec
Points: 1,040,000, Insert Time: 0.1568 sec, Total Search Time: 0.051547 sec, Total Time: 0.2083 sec
Points: 1,050,000, Insert Time: 0.1660 sec, Total Search Time: 0.059769 sec, Total Time: 0.2257 sec
Points: 1,060,000, Insert Time: 0.1807 sec, Total Search Time: 0.298555 sec, Total Time: 0.4792 sec
Points: 1,160,000, Insert Time: 1.6500 sec, Total Search Time: 0.084871 sec, Total Time: 1.7349 sec
Points: 1,170,000, Insert Time: 0.1710 sec, Total Search Time: 0.093732 sec, Total Time: 0.2647 sec
Points: 1,270,000, Insert Time: 1.6733


**-----VP Hybrid log Run 3--------**

In [61]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0251 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3981.19it/s]


Total Search Time: 0.028759 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 50432.98it/s]


Insert Time: 0.2004 sec


...

Total Search Time: 2.075207 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 58065.28it/s]


Insert Time: 1.7243 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 214.08it/s]

Total Search Time: 0.763361 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0251 sec, Total Search Time: 0.028759 sec, Total Time: 0.0538 sec
Points: 20,000, Insert Time: 0.2004 sec, Total Search Time: 0.050338 sec, Total Time: 0.2507 sec
Points: 30,000, Insert Time: 0.0836 sec, Total Search Time: 0.022239 sec, Total Time: 0.1059 sec
Points: 1,030,000, Build Time: 4.9239 sec, Total Search Time: 0.021069 sec, Total Time: 4.9450 sec
Points: 1,040,000, Insert Time: 0.1492 sec, Total Search Time: 0.040504 sec, Total Time: 0.1898 sec
Points: 1,050,000, Insert Time: 0.1613 sec, Total Search Time: 0.054218 sec, Total Time: 0.2155 sec
Points: 1,060,000, Insert Time: 0.2033 sec, Total Search Time: 0.263921 sec, Total Time: 0.4672 sec
Points: 1,160,000, Insert Time: 1.6931 sec, Total Search Time: 0.079154 sec, Total Time: 1.7723 sec
Points: 1,170,000, Insert Time: 0.1698 sec, Total Search Time: 0.092088 sec, Total Time: 0.2619 sec
Points: 1,270,000, Insert Time: 1.6878

**--------vp hybrid log analysis----------**

In [64]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 20.7393',
    'Total Time:: 20.2876',
    'Total Time:: 20.2212'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 20.4160 sec, Std Dev: 0.2819 sec, CV: 1.38%


-------------------------

**-----VP hybrid Log Ratio Run 1-------**

In [52]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0197 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3697.77it/s]


Total Search Time: 0.030612 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 121871.47it/s]


Insert Time: 0.0840 sec


...

Total Search Time: 2.713288 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59133.47it/s]


Insert Time: 1.6929 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 175.23it/s]

Total Search Time: 0.820127 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0197 sec, Total Search Time: 0.030612 sec, Total Time: 0.0503 sec
Points: 20,000, Insert Time: 0.0840 sec, Total Search Time: 0.165232 sec, Total Time: 0.2492 sec
Points: 30,000, Insert Time: 0.1008 sec, Total Search Time: 0.023591 sec, Total Time: 0.1244 sec
Points: 1,030,000, Build Time: 4.5716 sec, Total Search Time: 0.006720 sec, Total Time: 4.5783 sec
Points: 1,040,000, Insert Time: 0.1453 sec, Total Search Time: 0.033156 sec, Total Time: 0.1784 sec
Points: 1,050,000, Insert Time: 0.1662 sec, Total Search Time: 0.049833 sec, Total Time: 0.2161 sec
Points: 1,060,000, Insert Time: 0.1699 sec, Total Search Time: 0.247469 sec, Total Time: 0.4173 sec
Points: 1,160,000, Insert Time: 1.6988 sec, Total Search Time: 0.081436 sec, Total Time: 1.7803 sec
Points: 1,170,000, Insert Time: 0.1685 sec, Total Search Time: 0.085707 sec, Total Time: 0.2542 sec
Points: 1,270,000, Insert Time: 1.6792

**------VP Hybrid Log Ratio Run 2---------**

In [57]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0147 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3762.99it/s]


Total Search Time: 0.029883 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 121890.95it/s]


Insert Time: 0.0841 sec


...

Total Search Time: 2.586060 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 59086.97it/s]


Insert Time: 1.6947 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 183.43it/s]

Total Search Time: 0.810954 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0147 sec, Total Search Time: 0.029883 sec, Total Time: 0.0446 sec
Points: 20,000, Insert Time: 0.0841 sec, Total Search Time: 0.172585 sec, Total Time: 0.2567 sec
Points: 30,000, Insert Time: 0.0873 sec, Total Search Time: 0.023037 sec, Total Time: 0.1103 sec
Points: 1,030,000, Build Time: 4.2525 sec, Total Search Time: 0.017602 sec, Total Time: 4.2701 sec
Points: 1,040,000, Insert Time: 0.1512 sec, Total Search Time: 0.049972 sec, Total Time: 0.2012 sec
Points: 1,050,000, Insert Time: 0.1699 sec, Total Search Time: 0.060218 sec, Total Time: 0.2301 sec
Points: 1,060,000, Insert Time: 0.1666 sec, Total Search Time: 0.295289 sec, Total Time: 0.4619 sec
Points: 1,160,000, Insert Time: 1.7163 sec, Total Search Time: 0.092490 sec, Total Time: 1.8088 sec
Points: 1,170,000, Insert Time: 0.1686 sec, Total Search Time: 0.094974 sec, Total Time: 0.2635 sec
Points: 1,270,000, Insert Time: 1.6709

**------VP Hybrid Log Ratio Run 3----------**

In [62]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0140 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 4090.61it/s]


Total Search Time: 0.028062 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 44593.97it/s]


Insert Time: 0.2268 sec


...

Total Search Time: 2.065416 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 60165.66it/s]


Insert Time: 1.6638 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 210.75it/s]

Total Search Time: 0.781538 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0140 sec, Total Search Time: 0.028062 sec, Total Time: 0.0420 sec
Points: 20,000, Insert Time: 0.2268 sec, Total Search Time: 0.051454 sec, Total Time: 0.2783 sec
Points: 30,000, Insert Time: 0.0879 sec, Total Search Time: 0.022288 sec, Total Time: 0.1102 sec
Points: 1,030,000, Build Time: 4.8122 sec, Total Search Time: 0.020615 sec, Total Time: 4.8328 sec
Points: 1,040,000, Insert Time: 0.1479 sec, Total Search Time: 0.041759 sec, Total Time: 0.1897 sec
Points: 1,050,000, Insert Time: 0.1623 sec, Total Search Time: 0.056162 sec, Total Time: 0.2184 sec
Points: 1,060,000, Insert Time: 0.1716 sec, Total Search Time: 0.257601 sec, Total Time: 0.4292 sec
Points: 1,160,000, Insert Time: 1.6810 sec, Total Search Time: 0.081311 sec, Total Time: 1.7623 sec
Points: 1,170,000, Insert Time: 0.1710 sec, Total Search Time: 0.101024 sec, Total Time: 0.2720 sec
Points: 1,270,000, Insert Time: 1.6604

**------vp hybrid log ratio analysis----------**

In [63]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 20.6584',
    'Total Time:: 20.3176',
    'Total Time:: 19.9939'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 20.3233 sec, Std Dev: 0.3323 sec, CV: 1.64%


-----------------------------

**-------VP Tree Epsilon Dynamic-------**

In [66]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

**--------vp dyn sqrt run 1--------**

In [70]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0119 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3433.06it/s]


Total Search Time: 0.032546 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 42529.26it/s]


Insert Time: 0.2374 sec


...

Total Search Time: 2.730182 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 66274.96it/s]


Insert Time: 1.5113 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 167.61it/s]

Total Search Time: 0.933517 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0119 sec, Total Search Time: 0.032546 sec, Total Time: 0.0444 sec
Points: 20,000, Insert Time: 0.2374 sec, Total Search Time: 0.059433 sec, Total Time: 0.2969 sec
Points: 30,000, Build Time: 0.0535 sec, Total Search Time: 0.019462 sec, Total Time: 0.0730 sec
Points: 1,030,000, Build Time: 3.8020 sec, Total Search Time: 0.005206 sec, Total Time: 3.8072 sec
Points: 1,040,000, Insert Time: 0.1349 sec, Total Search Time: 0.029230 sec, Total Time: 0.1641 sec
Points: 1,050,000, Insert Time: 0.1363 sec, Total Search Time: 0.047018 sec, Total Time: 0.1833 sec
Points: 1,060,000, Insert Time: 0.1442 sec, Total Search Time: 0.246729 sec, Total Time: 0.3909 sec
Points: 1,160,000, Insert Time: 1.4685 sec, Total Search Time: 0.092554 sec, Total Time: 1.5610 sec
Points: 1,170,000, Insert Time: 0.1542 sec, Total Search Time: 0.090249 sec, Total Time: 0.2445 sec
Points: 1,270,000, Insert Time: 1.3710 

**----------vp dyn sqrt run 2--------**

In [75]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0214 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3288.05it/s]


Total Search Time: 0.035005 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 41020.21it/s]


Insert Time: 0.2462 sec


...

Total Search Time: 2.989606 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 72179.54it/s]


Insert Time: 1.3873 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 150.38it/s]

Total Search Time: 0.983105 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0214 sec, Total Search Time: 0.035005 sec, Total Time: 0.0565 sec
Points: 20,000, Insert Time: 0.2462 sec, Total Search Time: 0.058544 sec, Total Time: 0.3047 sec
Points: 30,000, Build Time: 0.0567 sec, Total Search Time: 0.019614 sec, Total Time: 0.0764 sec
Points: 1,030,000, Build Time: 3.5949 sec, Total Search Time: 0.010203 sec, Total Time: 3.6051 sec
Points: 1,040,000, Insert Time: 0.1337 sec, Total Search Time: 0.034775 sec, Total Time: 0.1685 sec
Points: 1,050,000, Insert Time: 0.1375 sec, Total Search Time: 0.052319 sec, Total Time: 0.1898 sec
Points: 1,060,000, Insert Time: 0.1379 sec, Total Search Time: 0.275820 sec, Total Time: 0.4137 sec
Points: 1,160,000, Insert Time: 1.4198 sec, Total Search Time: 0.081660 sec, Total Time: 1.5015 sec
Points: 1,170,000, Insert Time: 0.1346 sec, Total Search Time: 0.102476 sec, Total Time: 0.2371 sec
Points: 1,270,000, Insert Time: 1.3507 

**--------vp dyn sqrt run 3-------**

In [80]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0193 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 695.99it/s]


Total Search Time: 0.146969 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 106492.80it/s]


Insert Time: 0.0961 sec


...

Total Search Time: 2.516403 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 68020.52it/s]


Insert Time: 1.4718 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 191.46it/s]

Total Search Time: 0.859580 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0193 sec, Total Search Time: 0.146969 sec, Total Time: 0.1662 sec
Points: 20,000, Insert Time: 0.0961 sec, Total Search Time: 0.051472 sec, Total Time: 0.1476 sec
Points: 30,000, Build Time: 0.1941 sec, Total Search Time: 0.016478 sec, Total Time: 0.2106 sec
Points: 1,030,000, Build Time: 3.9769 sec, Total Search Time: 0.012475 sec, Total Time: 3.9894 sec
Points: 1,040,000, Insert Time: 0.1399 sec, Total Search Time: 0.031022 sec, Total Time: 0.1709 sec
Points: 1,050,000, Insert Time: 0.1550 sec, Total Search Time: 0.043565 sec, Total Time: 0.1986 sec
Points: 1,060,000, Insert Time: 0.1482 sec, Total Search Time: 0.225573 sec, Total Time: 0.3738 sec
Points: 1,160,000, Insert Time: 1.4165 sec, Total Search Time: 0.088476 sec, Total Time: 1.5050 sec
Points: 1,170,000, Insert Time: 0.1570 sec, Total Search Time: 0.090910 sec, Total Time: 0.2479 sec
Points: 1,270,000, Insert Time: 1.4054 

**-----vp dyn sqrt analysis-------**

In [85]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 18.7628',
    'Total Time:: 18.8146',
    'Total Time:: 18.6263'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 18.7346 sec, Std Dev: 0.0973 sec, CV: 0.52%


------------------------------

**-----VP  Dynamic Log  Run 1------**

In [71]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0214 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 670.89it/s]


Total Search Time: 0.152291 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 118362.46it/s]


Insert Time: 0.0863 sec


...

Total Search Time: 2.758894 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 65845.03it/s]


Insert Time: 1.5207 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 169.57it/s]

Total Search Time: 0.910225 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0214 sec, Total Search Time: 0.152291 sec, Total Time: 0.1737 sec
Points: 20,000, Insert Time: 0.0863 sec, Total Search Time: 0.059479 sec, Total Time: 0.1458 sec
Points: 30,000, Insert Time: 0.0841 sec, Total Search Time: 0.024515 sec, Total Time: 0.1087 sec
Points: 1,030,000, Build Time: 3.8442 sec, Total Search Time: 0.005829 sec, Total Time: 3.8500 sec
Points: 1,040,000, Insert Time: 0.1351 sec, Total Search Time: 0.029229 sec, Total Time: 0.1643 sec
Points: 1,050,000, Insert Time: 0.1343 sec, Total Search Time: 0.050873 sec, Total Time: 0.1852 sec
Points: 1,060,000, Insert Time: 0.1355 sec, Total Search Time: 0.241366 sec, Total Time: 0.3769 sec
Points: 1,160,000, Insert Time: 1.4533 sec, Total Search Time: 0.090566 sec, Total Time: 1.5439 sec
Points: 1,170,000, Insert Time: 0.1585 sec, Total Search Time: 0.088984 sec, Total Time: 0.2475 sec
Points: 1,270,000, Insert Time: 1.3541

**--------VP Dynamic log Run 2--------**

In [76]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0219 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 655.89it/s]


Total Search Time: 0.156353 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 119343.86it/s]


Insert Time: 0.0860 sec


...

Total Search Time: 2.991048 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 73343.47it/s]


Insert Time: 1.3654 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 147.00it/s]

Total Search Time: 0.993470 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0219 sec, Total Search Time: 0.156353 sec, Total Time: 0.1783 sec
Points: 20,000, Insert Time: 0.0860 sec, Total Search Time: 0.059678 sec, Total Time: 0.1457 sec
Points: 30,000, Insert Time: 0.0852 sec, Total Search Time: 0.026748 sec, Total Time: 0.1119 sec
Points: 1,030,000, Build Time: 3.5455 sec, Total Search Time: 0.010050 sec, Total Time: 3.5556 sec
Points: 1,040,000, Insert Time: 0.1320 sec, Total Search Time: 0.036134 sec, Total Time: 0.1682 sec
Points: 1,050,000, Insert Time: 0.1396 sec, Total Search Time: 0.053541 sec, Total Time: 0.1932 sec
Points: 1,060,000, Insert Time: 0.1315 sec, Total Search Time: 0.274384 sec, Total Time: 0.4059 sec
Points: 1,160,000, Insert Time: 1.3747 sec, Total Search Time: 0.093817 sec, Total Time: 1.4685 sec
Points: 1,170,000, Insert Time: 0.1334 sec, Total Search Time: 0.106004 sec, Total Time: 0.2395 sec
Points: 1,270,000, Insert Time: 1.3415

**------VP Dynamic log Run 3----------**

In [81]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.1662 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3874.79it/s]


Total Search Time: 0.030081 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 116090.52it/s]


Insert Time: 0.0886 sec


...

Total Search Time: 2.450485 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69599.04it/s]


Insert Time: 1.4385 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 189.95it/s]

Total Search Time: 0.867519 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.1662 sec, Total Search Time: 0.030081 sec, Total Time: 0.1963 sec
Points: 20,000, Insert Time: 0.0886 sec, Total Search Time: 0.050953 sec, Total Time: 0.1395 sec
Points: 30,000, Insert Time: 0.0867 sec, Total Search Time: 0.024268 sec, Total Time: 0.1110 sec
Points: 1,030,000, Build Time: 4.1108 sec, Total Search Time: 0.012646 sec, Total Time: 4.1235 sec
Points: 1,040,000, Insert Time: 0.1380 sec, Total Search Time: 0.031308 sec, Total Time: 0.1693 sec
Points: 1,050,000, Insert Time: 0.1435 sec, Total Search Time: 0.040107 sec, Total Time: 0.1836 sec
Points: 1,060,000, Insert Time: 0.1407 sec, Total Search Time: 0.209167 sec, Total Time: 0.3498 sec
Points: 1,160,000, Insert Time: 1.4318 sec, Total Search Time: 0.081218 sec, Total Time: 1.5131 sec
Points: 1,170,000, Insert Time: 0.1548 sec, Total Search Time: 0.086067 sec, Total Time: 0.2408 sec
Points: 1,270,000, Insert Time: 1.4491

**---------vp dyn log analysis---------**

In [84]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 18.7614',
    'Total Time:: 18.7791',
    'Total Time:: 18.6469'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 18.7291 sec, Std Dev: 0.0718 sec, CV: 0.38%


------------------------------------

**-----VP  Dynamic Log Ratio Run 1-----**

In [72]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0207 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 562.85it/s]


Total Search Time: 0.181000 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 120499.66it/s]


Insert Time: 0.0857 sec


...

Total Search Time: 2.724261 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 65937.00it/s]


Insert Time: 1.5183 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 169.11it/s]

Total Search Time: 0.900349 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0207 sec, Total Search Time: 0.181000 sec, Total Time: 0.2017 sec
Points: 20,000, Insert Time: 0.0857 sec, Total Search Time: 0.059212 sec, Total Time: 0.1449 sec
Points: 30,000, Insert Time: 0.0861 sec, Total Search Time: 0.024758 sec, Total Time: 0.1108 sec
Points: 1,030,000, Build Time: 3.8233 sec, Total Search Time: 0.005109 sec, Total Time: 3.8284 sec
Points: 1,040,000, Insert Time: 0.1345 sec, Total Search Time: 0.028576 sec, Total Time: 0.1631 sec
Points: 1,050,000, Insert Time: 0.1357 sec, Total Search Time: 0.046071 sec, Total Time: 0.1817 sec
Points: 1,060,000, Insert Time: 0.1365 sec, Total Search Time: 0.237836 sec, Total Time: 0.3744 sec
Points: 1,160,000, Insert Time: 1.4569 sec, Total Search Time: 0.083015 sec, Total Time: 1.5399 sec
Points: 1,170,000, Insert Time: 0.1520 sec, Total Search Time: 0.097793 sec, Total Time: 0.2498 sec
Points: 1,270,000, Insert Time: 1.3902

**--------VP Dyn Log Ratio Run 2-------**

In [77]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0115 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 3459.99it/s]


Total Search Time: 0.032372 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 42230.46it/s]


Insert Time: 0.2391 sec


...

Total Search Time: 2.991327 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 72500.94it/s]


Insert Time: 1.3818 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 152.01it/s]

Total Search Time: 0.968544 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0115 sec, Total Search Time: 0.032372 sec, Total Time: 0.0439 sec
Points: 20,000, Insert Time: 0.2391 sec, Total Search Time: 0.059561 sec, Total Time: 0.2987 sec
Points: 30,000, Insert Time: 0.0858 sec, Total Search Time: 0.025644 sec, Total Time: 0.1115 sec
Points: 1,030,000, Build Time: 3.5378 sec, Total Search Time: 0.010186 sec, Total Time: 3.5480 sec
Points: 1,040,000, Insert Time: 0.1306 sec, Total Search Time: 0.035314 sec, Total Time: 0.1659 sec
Points: 1,050,000, Insert Time: 0.1345 sec, Total Search Time: 0.051449 sec, Total Time: 0.1859 sec
Points: 1,060,000, Insert Time: 0.1327 sec, Total Search Time: 0.277986 sec, Total Time: 0.4107 sec
Points: 1,160,000, Insert Time: 1.3828 sec, Total Search Time: 0.085997 sec, Total Time: 1.4688 sec
Points: 1,170,000, Insert Time: 0.1378 sec, Total Search Time: 0.109095 sec, Total Time: 0.2469 sec
Points: 1,270,000, Insert Time: 1.3429

**-------VP Dyn Log Ratio Run 3--------**

In [82]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0211 sec


Querying batch 1: 100%|██████████| 100/100 [00:00<00:00, 651.05it/s]


Total Search Time: 0.156946 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 110594.41it/s]


Insert Time: 0.0924 sec


...

Total Search Time: 2.463800 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69621.43it/s]


Insert Time: 1.4387 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 185.85it/s]

Total Search Time: 0.855551 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0211 sec, Total Search Time: 0.156946 sec, Total Time: 0.1780 sec
Points: 20,000, Insert Time: 0.0924 sec, Total Search Time: 0.054319 sec, Total Time: 0.1467 sec
Points: 30,000, Insert Time: 0.0931 sec, Total Search Time: 0.023542 sec, Total Time: 0.1167 sec
Points: 1,030,000, Build Time: 4.1741 sec, Total Search Time: 0.013715 sec, Total Time: 4.1878 sec
Points: 1,040,000, Insert Time: 0.1378 sec, Total Search Time: 0.030581 sec, Total Time: 0.1684 sec
Points: 1,050,000, Insert Time: 0.1416 sec, Total Search Time: 0.044419 sec, Total Time: 0.1860 sec
Points: 1,060,000, Insert Time: 0.1397 sec, Total Search Time: 0.212823 sec, Total Time: 0.3526 sec
Points: 1,160,000, Insert Time: 1.4348 sec, Total Search Time: 0.084836 sec, Total Time: 1.5196 sec
Points: 1,170,000, Insert Time: 0.1581 sec, Total Search Time: 0.096983 sec, Total Time: 0.2550 sec
Points: 1,270,000, Insert Time: 1.4163

**--------vp dyn log ratio analysis--------**

In [83]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 18.7698',
    'Total Time:: 18.7624',
    'Total Time:: 18.7667'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 18.7663 sec, Std Dev: 0.0037 sec, CV: 0.02%
